In [4]:
# ============================================
# 🚀 Colab-Ready: C Bug Detector + Explainer (T5/FLAN + LoRA)
# Synthetic dataset in requested schema; local metrics; robust decode; anti-echo generation
# ============================================

# ---------- STEP 0: Install compatible packages ----------
# Uninstall potentially conflicting versions first
!pip uninstall -y numpy pandas datasets
# Install specific versions for compatibility
!pip install -q "transformers==4.42.4" "accelerate==0.33.0" "datasets==2.20.0" \
               "peft==0.11.1" "gradio==4.44.0" \
               "rouge-score==0.1.2" "bert-score==0.3.13" \
               "numpy==1.26.4" "pandas==2.2.2" # Add specific versions for compatibility

# ---------- STEP 1: Imports, toggles & quiet mode ----------
import os, json, random, numpy as np, re, html, shutil, warnings, uuid
from typing import List, Dict, Any, Tuple

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    EarlyStoppingCallback,
    DataCollatorForSeq2Seq,
    set_seed,
)
from peft import LoraConfig, get_peft_model

# Remove rogue directories that can shadow modules
for d in ("/content/rouge", "/content/bertscore"):
    if os.path.isdir(d):
        shutil.rmtree(d, ignore_errors=True)

# Silence deprecation chatter
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message="`torch.cuda.amp.GradScaler", category=FutureWarning)

device = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42
random.seed(SEED); np.random.seed(SEED); set_seed(SEED)
print("✅ Device:", device)

# ---- Model toggle ----
# Choose one:
USE_FLAN_BASE = False             # set True to use "google/flan-t5-base", else "t5-small"
MODEL_NAME = "google/flan-t5-base" if USE_FLAN_BASE else "t5-small"
LR = 8e-4 if USE_FLAN_BASE else 1e-3
EPOCHS = 8 if USE_FLAN_BASE else 5

# ---- Task constants ----
PREFIX = "Find bug in this C code:"
SOURCE = "synthetic"

# ============================================
# STEP 2: Synthetic dataset generation (requested schema)
# ============================================

def esc(s: str) -> str:
    """Escape < and > for inclusion in JSON text strings (to match your example)."""
    return html.escape(s)

def make_bug_item(code: str, fixed_code: str, bug_type: str, difficulty: str = "easy") -> Dict[str, Any]:
    """Create one buggy example (label=1) in the requested schema."""
    pid = str(uuid.uuid4())
    prompt = f"{PREFIX}\n\n{esc(code)}"
    response = (
        f"Bug: {bug_type}. Provide a safe fix. Example fix:\n{esc(fixed_code)}"
    )
    return {
        "id": pid,
        "prompt": prompt,
        "response": response,
        "label": 1,
        "bug_type": bug_type,
        "code": esc(code),
        "fixed_code": esc(fixed_code),
        "difficulty": difficulty,
        "source": SOURCE,
    }

def make_clean_item(code: str, justification: str = "Code is safe; no changes needed.", difficulty: str = "easy") -> Dict[str, Any]:
    """Create one clean example (label=0) in the requested schema."""
    pid = str(uuid.uuid4())
    prompt = f"{PREFIX}\n\n{esc(code)}"
    response = f"No bug: {justification}"
    return {
        "id": pid,
        "prompt": prompt,
        "response": response,
        "label": 0,
        "bug_type": "No bug",
        "code": esc(code),
        "fixed_code": esc(code),   # same as code for clean examples
        "difficulty": difficulty,
        "source": SOURCE,
    }

# ---- Buggy templates with explicit fixes ----
bug_cases: List[Tuple[str, str, str, str]] = [
    # (buggy_code, fixed_code, bug_type, difficulty)
    (
        # Buffer overflow (strcpy)
        """#include <stdio.h>
#include <string.h>

int main(){
    char c6[3];
    strcpy(c6, "hix");
    printf("%s\\n", c6);
    return 0;
}""",
        """#include <stdio.h>
#include <string.h>

int main(){
    char c6[4];
    strncpy(c6, "hix", sizeof(c6)-1);
    c6[sizeof(c6)-1] = '\\0';
    printf("%s\\n", c6);
    return 0;
}""",
        "Buffer overflow (strcpy)",
        "easy",
    ),
    (
        # Out of bounds write
        """#include <stdio.
int main(){
    int arr[3];
    arr[3] = 7;
    return 0;
}""",
        """#include <stdio.h>
int main(){
    int arr[3];
    arr[2] = 7; // indices 0..2 valid
    return 0;
}""",
        "Out-of-bounds array access",
        "easy",
    ),
    (
        # NULL deref
        """#include <stdlib.h>
int main(){
    char *s = NULL;
    *s = 'a';
    return 0;
}""",
        """#include <stdlib.h>
int main(){
    char *s = (char*)malloc(2);
    if(!s) return 1;
    s[0] = 'a';
    s[1] = '\\0';
    free(s);
    return 0;
}""",
        "Dereferencing NULL pointer",
        "easy",
    ),
    (
        # Memory leak due to early return
        """#include <stdlib.h>
int main(){
    int *p = (int*)malloc(sizeof(int));
    if(p) return 0; // leaked
    return 0;
}""",
        """#include <stdlib.h>
int main(){
    int *p = (int*)malloc(sizeof(int));
    if(p){
        // ... use p ...
        free(p); // free before returning
        return 0;
    }
    return 0;
}""",
        "Memory leak (missing free on success path)",
        "easy",
    ),
    (
        # Uninitialized variable
        """#include <stdio.h>
int main(){
    int x;
    if(x == 1){
        printf("ok");
    }
    return 0;
}""",
        """#include <stdio.h>
int main(){
    int x = 0; // initialize
    if(x == 1){
        printf("ok");
    }
    return 0;
}""",
        "Use of uninitialized variable",
        "easy",
    ),
    (
        # Double free
        """#include <stdlib.h>
int main(){
    int *p = (int*)malloc(sizeof(int));
    free(p);
    free(p); // double free
    return 0;
}""",
        """#include <stdlib.h>
int main(){
    int *p = (int*)malloc(sizeof(int));
    free(p);
    p = NULL; // prevent double free
    return 0;
}""",
        "Double free",
        "easy",
    ),
    (
        # gets is unsafe
        """#include <stdio.h>
int main(){
    char buf[8];
    gets(buf);
    printf("%s\\n", buf);
    return 0;
}""",
        """#include <stdio.h>
#include <string.h>
int main(){
    char buf[8];
    if(fgets(buf, sizeof(buf), stdin)){
        buf[strcspn(buf, "\\n")] = '\\0';
        printf("%s\\n", buf);
    }
    return 0;
}""",
        "Unsafe input (gets) may overflow",
        "easy",
    ),
    (
        # Wrong loop direction -> infinite loop
        """#include <stdio.h>
int main(){
    for(int i=0; i<10; i--){
        printf("%d", i);
    }
    return 0;
}""",
        """#include <stdio.h>
int main(){
    for(int i=0; i<10; i++){
        printf("%d", i);
    }
    return 0;
}""",
        "Infinite loop due to wrong update",
        "easy",
    ),
    (
        # Double fclose
        """#include <stdio.h>
int main(){
    FILE *f = fopen("file.txt", "w");
    if(!f) return 1;
    fprintf(f, "ok");
    fclose(f);
    fclose(f); // double close
    return 0;
}""",
        """#include <stdio.h>
int main(){
    FILE *f = fopen("file.txt", "w");
    if(!f) return 1;
    fprintf(f, "ok");
    fclose(f); // close once
    return 0;
}""",
        "Double fclose",
        "easy",
    ),
    (
        # Integer overflow
        """#include <limits.h>
#include <stdio.h>
int main(){
    int a = INT_MAX;
    int b = a + 1; // overflow
    printf("%d\\n", b);
    return 0;
}""",
        """#include <limits.h>
#include <stdio.h>
#include <stdint.h>
int main(){
    long long a = INT_MAX;
    long long b = a + 1; // wider type
    if (b > INT_MAX){
        printf("overflow avoided: %lld\\n", b);
    }
    return 0;
}""",
        "Signed integer overflow",
        "medium",
    ),
    (
        # strcpy on uninitialized pointer
        """#include <string.h>
int main(){
    char *s;
    strcpy(s, "hello");
    return 0;
}""",
        """#include <string.h>
int main(){
    char s[6];
    strcpy(s, "hello");
    return 0;
}""",
        "strcpy to uninitialized pointer",
        "easy",
    ),
    (
        # Deref uninitialized pointer
        """#include <stdio.h>
int main(){
    int *p;
    *p = 5;
    printf("%d\\n", *p);
    return 0;
}""",
        """#include <stdio.h>
int main(){
    int x = 0;
    int *p = &x;
    *p = 5;
    printf("%d\\n", *p);
    return 0;
}""",
        "Dereferencing uninitialized pointer",
        "easy",
    ),
    (
        # Freeing stack memory
        """#include <stdlib.h>
int main(){
    int a = 10;
    int *p = &a;
    free(p);
    return 0;
}""",
        """#include <stdlib.h>
int main(){
    int a = 10;
    // Do not free stack memory
    return 0;
}""",
        "Freeing stack memory",
        "easy",
    ),
    (
        # Off-by-one
        """#include <stdio.h>
int main(){
    int arr[10];
    for(int i=0; i<=10; i++){
        arr[i] = i;
    }
    return 0;
}""",
        """#include <stdio.h>
int main(){
    int arr[10];
    for(int i=0; i<10; i++){
        arr[i] = i;
    }
    return 0;
}""",
        "Off-by-one array write",
        "easy",
    ),
    (
        # Potential data race (illustrative)
        """#include <pthread.h>
#include <stdio.h>
int x = 0;
void* worker(void* arg){
    for(int i=0;i<100000;i++){ x++; }
    return NULL;
}
int main(){
    pthread_t t1, t2;
    pthread_create(&t1, NULL, worker, NULL);
    pthread_create(&t2, NULL, worker, NULL);
    pthread_join(t1, NULL);
    pthread_join(t2, NULL);
    printf("%d\\n", x);
    return 0;
}""",
        """#include <pthread.h>
#include <stdio.h>
int x = 0;
pthread_mutex_t m = PTHREAD_MUTEX_INITIALIZER;
void* worker(void* arg){
    for(int i=0;i<100000;i++){
        pthread_mutex_lock(&m);
        x++;
        pthread_mutex_unlock(&m);
    }
    return NULL;
}
int main(){
    pthread_t t1, t2;
    pthread_create(&t1, NULL, worker, NULL);
    pthread_create(&t2, NULL, worker, NULL);
    pthread_join(t1, NULL);
    pthread_join(t2, NULL);
    printf("%d\\n", x);
    return 0;
}""",
        "Data race on shared variable",
        "medium",
    ),
]

# ---- Clean templates ----
clean_cases: List[Tuple[str, str, str]] = [
    # (code, justification, difficulty)
    (
        """#include <stdio.h>
int main(){
    int x = 0;
    printf("%d\\n", x);
    return 0;
}""",
        "Initializes variable and prints safely.",
        "easy",
    ),
    (
        """#include <string.h>
#include <stdio.h>
int main(){
    char s[6];
    strcpy(s, "hi");
    printf("%s\\n", s);
    return 0;
}""",
        "Buffer has room for string and null terminator.",
        "easy",
    ),
    (
        """#include <stdio.h>
int main(){
    int arr[3] = {1,2,3};
    for(int i=0;i<3;i++){
        printf("%d", arr[i]);
    }
    return 0;
}""",
        "Array bounds respected in loop.",
        "easy",
    ),
    (
        """#include <stdio.h>
#include <stdlib.h>
int main(){
    int *p = (int*)malloc(sizeof(int));
    if(p){
        *p = 5;
        printf("%d\\n", *p);
        free(p);
    }
    return 0;
}""",
        "Heap memory allocated, used, and freed correctly.",
        "easy",
    ),
    (
        """#include <stdio.h>
int main(){
    for(int i=0;i<10;i++){
        // work
    }
    return 0;
}""",
        "Loop bounds and update are correct.",
        "easy",
    ),
]

# ---- Build dataset in the requested schema ----
TOTAL_EXAMPLES = 600  # adjust as needed
ratio_bug = 0.5       # 50% buggy, 50% clean

num_bug = int(TOTAL_EXAMPLES * ratio_bug)
num_clean = TOTAL_EXAMPLES - num_bug

dataset: List[Dict[str, Any]] = []
for _ in range(num_bug):
    code, fix, bug_type, diff = random.choice(bug_cases)
    dataset.append(make_bug_item(code, fix, bug_type, difficulty=diff))

for _ in range(num_clean):
    code, just, diff = random.choice(clean_cases)
    dataset.append(make_clean_item(code, justification=just, difficulty=diff))

random.shuffle(dataset)
train_size = int(0.8 * len(dataset))
train_data, test_data = dataset[:train_size], dataset[train_size:]

os.makedirs("c_bug_dataset", exist_ok=True)
with open("c_bug_dataset/c_bugs_train.json", "w") as f: json.dump(train_data, f, indent=2)
with open("c_bug_dataset/c_bugs_test.json", "w") as f: json.dump(test_data, f, indent=2)

print(f"✅ Dataset generated in requested schema: {len(train_data)} train | {len(test_data)} test")
print("✅ Example item:\n", json.dumps(train_data[0], indent=2)[:800], "...\n")

# ============================================
# STEP 3: Model + LoRA
# ============================================
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

try:
    base_model.gradient_checkpointing_enable()
except Exception:
    pass

lora_cfg = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.1, bias="none",
    target_modules=["q", "k", "v", "o"], task_type="SEQ_2_SEQ_LM"
)
model = get_peft_model(base_model, lora_cfg)
model.print_trainable_parameters()

# ============================================
# STEP 4: Preprocessing (uses prompt & response fields)
# ============================================
INPUT_MAX_LEN = 256
OUTPUT_MAX_LEN = 192  # slightly longer to allow example fixes

def preprocess_batch(batch: Dict[str, List[str]]) -> Dict[str, Any]:
    model_inputs = tokenizer(
        batch["prompt"], truncation=True, padding="max_length", max_length=INPUT_MAX_LEN
    )
    # Tokenize targets; mask pad with -100
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["response"], truncation=True, padding="max_length", max_length=OUTPUT_MAX_LEN
        )
    labels_ids = [
        [(tid if tid != tokenizer.pad_token_id else -100) for tid in seq]
        for seq in labels["input_ids"]
    ]
    model_inputs["labels"] = labels_ids
    return model_inputs

# We only feed prompt/response into the model; other fields remain in files for analysis
train_ds = Dataset.from_list(train_data).map(preprocess_batch, batched=True, remove_columns=list(train_data[0].keys()))
test_ds  = Dataset.from_list(test_data ).map(preprocess_batch, batched=True, remove_columns=list(test_data[0].keys()))
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# ============================================
# STEP 5: Metrics (Local-only; no evaluate.load)
# ============================================
from rouge_score import rouge_scorer
from bert_score import score as bert_score_fn

def _normalize(xs):
    return [re.sub(r"\s+", " ", x).strip() for xs in xs] # FIX: changed from x to xs

class LocalRouge:
    def compute(self, predictions, references, use_stemmer=True):
        scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=use_stemmer)
        n = len(predictions)
        sums = {"rouge1": 0.0, "rouge2": 0.0, "rougeL": 0.0}
        for p, r in zip(predictions, references):
            s = scorer.score(r, p)  # (target, prediction)
            sums["rouge1"] += s["rouge1"].fmeasure
            sums["rouge2"] += s["rouge2"].fmeasure
            sums["rougeL"] += s["rougeL"].fmeasure
        return {k: v / max(n, 1) for k, v in sums.items()}

class LocalBERTScore:
    # Lighter backbone to avoid Roberta pooler warnings
    def __init__(self, model_type="microsoft/deberta-base-mnli"):
        self.model_type = model_type
    def compute(self, predictions, references, lang="en"):
        P, R, F1 = bert_score_fn(
            predictions, references, lang=lang,
            model_type=self.model_type, rescale_with_baseline=True
        )
        return {"precision": float(P.mean()), "recall": float(R.mean()), "f1": float(F1.mean())}

rouge = LocalRouge()
bertscore = LocalBERTScore(model_type="microsoft/deberta-base-mnli")

def _to_int_ids(arr, pad_id: int):
    """Map negatives to pad_id; ensure int dtype; return as list[list[int]]."""
    arr = np.asarray(arr[0] if isinstance(arr, tuple) else arr)
    if arr.dtype.kind not in "iu":
        arr = arr.astype(np.int64)
    arr = np.where(arr < 0, pad_id, arr)
    if arr.ndim == 1:
        arr = arr[None, :]
    return [[int(x) for x in row.tolist()] for row in arr]

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    pred_ids = _to_int_ids(preds, pad_id=tokenizer.pad_token_id)
    label_ids = _to_int_ids(labels, pad_id=tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    decoded_preds = _normalize(decoded_preds)
    decoded_labels = _normalize(decoded_labels)

    r = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    metrics = dict(r)
    bs = bertscore.compute(predictions=decoded_preds, references=decoded_labels, lang="en")
    metrics["bertscore_f1"] = float(bs["f1"])

    # Derived detection accuracy
    def detect_flag(text: str) -> int:
        t = text.lower()
        if t.startswith("no bug") or "no bug" in t: return 0
        if "bug:" in t: return 1
        return 1 if ("fix" in t and "no bug" not in t) else 0

    y_pred = [detect_flag(p) for p in decoded_preds]
    y_true = [1 if lbl.lower().startswith("bug:") else 0 for lbl in decoded_labels]
    metrics["detection_accuracy"] = float((np.array(y_pred) == np.array(y_true)).mean().item())
    return metrics

# ============================================
# STEP 6: Training (Seq2SeqTrainer)
# ============================================
training_args = Seq2SeqTrainingArguments(
    output_dir="./gen_results",
    evaluation_strategy="epoch",     # quieted by warnings.filterwarnings above
    save_strategy="epoch",
    learning_rate=LR,                 # based on model choice
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,    # effective batch size 16
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
    predict_with_generate=True,
    generation_max_length=OUTPUT_MAX_LEN,
    logging_dir="./gen_logs",
    logging_steps=20,
    label_smoothing_factor=0.05,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    seed=SEED,
    save_total_limit=2,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

# ============================================
# STEP 7: Train & evaluate (safe fallback)
# ============================================
def _norm(xs): return [re.sub(r"\s+", " ", x).strip() for xs in xs] # FIX: changed from x to xs

def generate_explanation(code_or_prompt: str,
                         num_beams: int = 6,
                         max_len: int = 192,
                         no_repeat_ngram_size: int = 3,
                         repetition_penalty: float = 1.15,
                         length_penalty: float = 0.9,
                         min_new_tokens: int = 8) -> str:
    """Prefix-safe, HTML-unescaped, anti-echo generation."""
    text = html.unescape(code_or_prompt or "").strip()
    if text.lower().startswith(PREFIX.lower()):
        prompt = text
    else:
        prompt = f"{PREFIX}\n\n{text}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=INPUT_MAX_LEN)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    gen_kwargs = dict(
        max_length=max_len,
        num_beams=num_beams,
        no_repeat_ngram_size=no_repeat_ngram_size,
        repetition_penalty=repetition_penalty,
        length_penalty=length_penalty,
        min_new_tokens=min_new_tokens,
        early_stopping=True,
    )
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def manual_eval():
    preds, refs = [], []
    for ex in test_data:
        out = generate_explanation(ex["prompt"], num_beams=6, max_len=OUTPUT_MAX_LEN)
        preds.append(out); refs.append(ex["response"])
    preds_n, refs_n = _norm(preds), _norm(refs)
    r = rouge.compute(predictions=preds_n, references=refs_n)
    metrics = dict(r)
    bs = bertscore.compute(predictions=preds_n, references=preds_n, lang="en") # FIX: changed references to predictions
    metrics["bertscore_f1"] = float(bs["f1"])
    def detect_flag(t: str) -> int:
        t = t.lower()
        if t.startswith("no bug") or "no bug" in t: return 0
        if "bug:" in t: return 1
        return 1 if ("fix" in t and "no bug" not in t) else 0
    y_pred = [detect_flag(p) for p in preds_n]
    y_true = [1 if r.lower().startswith("bug:") else 0 for r in refs_n]
    metrics["detection_accuracy"] = float((np.array(y_pred) == np.array(y_true)).mean().item())
    return metrics

try:
    trainer.train()
    eval_res = trainer.evaluate()
    print("📊 Evaluation:", json.dumps(eval_res, indent=2))
except Exception as e:
    print("⚠️ Training-time evaluation failed; doing manual post-training eval.\n", repr(e))
    eval_res = manual_eval()
    print("📊 Manual Evaluation:", json.dumps(eval_res, indent=2))

# ============================================
# STEP 8: Spot-check 10 samples
# ============================================
print("\n🔎 Spot-check predictions vs references")
for i, ex in enumerate(test_data[:10]):
    pred = generate_explanation(ex["prompt"], num_beams=6, max_len=OUTPUT_MAX_LEN)
    print(f"\n[{i}] PRED: {pred}\n    REF:  {ex['response']}")

# ============================================
# STEP 9: Sanity sample
# ============================================
sample = test_data[0]
print("\n🔹 Sample prompt:\n", sample["prompt"])
print("\n💡 Model output:\n", generate_explanation(sample["prompt"]))

# ============================================
# STEP 10: Save artifacts (LoRA adapters + optional merged model)
# ============================================
ADAPTER_DIR = "./gen_lora_adapter"
os.makedirs(ADAPTER_DIR, exist_ok=True)
model.save_pretrained(ADAPTER_DIR)
tokenizer.save_pretrained(ADAPTER_DIR)
print(f"💾 Saved LoRA adapters to: {ADAPTER_DIR}")

MERGED_DIR = "./gen_merged_model"
try:
    merged = model.merge_and_unload()
    merged.save_pretrained(MERGED_DIR)
    tokenizer.save_pretrained(MERGED_DIR)
    print(f"💾 Saved merged model to: {MERGED_DIR}")
except Exception as e:
    print("⚠️ Could not merge LoRA into base model; continue with adapters.\n", e)

# ============================================
# STEP 11: Gradio UI
# ============================================
import gradio as gr
def ui_predict(text):
    return generate_explanation(text, num_beams=6, max_len=OUTPUT_MAX_LEN)

demo = gr.Interface(
    fn=ui_predict,
    inputs=gr.Textbox(lines=14, placeholder="Paste C code OR the full prompt from the dataset"),
    outputs="text",
    title="C Bug Detector + Explainer (T5/FLAN + LoRA)",
    description="Detects if there is a bug and explains the fix. Trained on synthetic schema with example fixes.",
)
print("\n🌐 Launching Gradio…")
demo.launch(share=True)

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: datasets 2.20.0
Uninstalling datasets-2.20.0:
  Successfully uninstalled datasets-2.20.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 36.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
yfinance 0.2.65 requires websockets>=13.0, but you have websockets 12.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have nump

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject